In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
# from open3d import JVisualizer 
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
# op3.visualization.webrtc_server.enable_webrtc()
from open3d.web_visualizer import draw
from VisulizerTools import *
from tqdm import tqdm_notebook
import time
from sklearn.cluster import DBSCAN


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [3]:
collector = RansacCollector(pcap_path=r'D:\LiDAR_Data\MidTown\Liberty\2021-12-11-17-0-0.pcap',output_file_path='D:\LiDAR_Data\MidTown\Liberty',update_frame_num=100)
collector.gen_tdmap()

Loading pcap...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 27.92it/s]


In [4]:
threshold_map = np.load(r'D:\LiDAR_Data\MidTown\Liberty\Calibration\bck_map.npy')

In [11]:
bck_path = r'D:\LiDAR_Data\MidTown\PCAP_BCK\Liberty'
output_path = r'D:\LiDAR_Data\MidTown\PCAP_BCK\Liberty'
pcaps_list = os.listdir(bck_path)
pcaps_list = [p for p in pcaps_list if 'pcap' in p.split('.')]
aggregated_maps_multiple = []
for i in range(len(pcaps_list)):
    collector = RansacCollector(pcap_path=os.path.join(bck_path,pcaps_list[i]),output_file_path=bck_path,update_frame_num=2000)
    collector.gen_tdmap()
    aggregated_maps_multiple.append(collector.aggregated_map[np.random.choice(np.arange(0,2000),size = int(6000/len(pcaps_list)) ,replace = False)])
collector.aggregated_map = np.concatenate(aggregated_maps_multiple,axis = 0)
collector.gen_thredmap(d = 0.5 ,thred_s = 0.54,N = 10,delta_thred = 1e-3,step = 0.1)
threshold_map = collector.thred_map

Loading pcap...


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:12<00:00, 27.69it/s]


Loading pcap...


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:11<00:00, 28.16it/s]


Loading pcap...


...

Loading pcap...


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:08<00:00, 29.25it/s]


Loading pcap...


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:08<00:00, 29.04it/s]


Generating Threshold Map


In [31]:
pcaps_list

['2021-12-10-3-30-0.pcap',
 '2021-12-11-1-0-0.pcap',
 '2021-12-11-11-30-0.pcap',
 '2021-12-11-4-0-0.pcap',
 '2021-12-12-5-0-0.pcap']

In [32]:
collector.aggregated_map = collector.aggregated_map[:]

In [29]:
collector.gen_thredmap(d = 1.2 ,thred_s = 0.5,N = 10,delta_thred = 1e-3,step = 0.1)
threshold_map = collector.thred_map

Generating Threshold Map


In [30]:
clustering_td = Raster_DBSCAN(window_size=(5,13),eps = 1.2, min_samples= 8, Td_map_szie=threshold_map.shape)   
Td_map = collector.aggregated_map[0]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_10')

In [ ]:
clustering_td = Raster_DBSCAN(window_size=(5,13),eps = 1.2, min_samples= 8, Td_map_szie=threshold_map.shape)   
Td_map = collector.aggregated_map[0]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)